<a href="https://colab.research.google.com/github/kutayoncuyilmaz/Thesis/blob/main/Twitter_Data_Cleaning_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Feb  3 10:30:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import random
import re

In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/SentiBert/training.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1")

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df = data.sample(frac=0.2,random_state=42)

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Delete the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Delete URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Just keep letters and important punctuation
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Remove additional spaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
df_clean = [clean_tweet(tweet) for tweet in df.text]

In [ ]:
df['text_final']=df_clean
df.drop('text',axis=1,inplace=True)

In [ ]:
df_labels = df.sentiment.values
df_labels[df_labels == 4] = 1

In [ ]:
df.rename(columns={'text_final':'text'},inplace=True)

In [ ]:
df['sentiment'].value_counts()

1    160506
0    159494
Name: sentiment, dtype: int64

In [ ]:
df.head()

,sentiment,text
541200,0,AHHH I HOPE YOUR OK!!!
750,0,cool i have no tweet apps for my razr
766711,0,i know just family drama. its lame.hey next t...
285055,0,School email won't open and I have geography s...
705995,0,upper airways problem


In [ ]:
from datetime import date
df.to_csv(f"/content/drive/MyDrive/SentiBert/df_{date.today()}.csv")